In [ ]:
!pip install -q --upgrade langchain-google-genai google-generativeai python-dotenv matplotlib pandas gradio

from langchain_google_genai import ChatGoogleGenerativeAI
from langchain.prompts import PromptTemplate
from langchain.output_parsers import PydanticOutputParser
from pydantic import BaseModel, Field, validator
from dotenv import load_dotenv
import os
import matplotlib.pyplot as plt
import pandas as pd
import gradio as gr

GOOGLE_API_KEY = "YOUR_API_KEY_HERE" #replace your api key here
os.environ["GOOGLE_API_KEY"] = GOOGLE_API_KEY

model = ChatGoogleGenerativeAI(model="gemini-1.5-flash", google_api_key=GOOGLE_API_KEY)

class SentimentAnalysis(BaseModel):
    sentiment: str = Field(description="POSITIVE, NEGATIVE, NEUTRAL, IRRELEVANT, HARMFUL, or SUGGESTIVE")
    emotion: str = Field(description="Dominant emotion such as Happy, Angry, Sad, Confused, etc.")
    rating: int = Field(ge=1, le=5, description="Rating between 1 (worst) to 5 (best)")

    @validator("sentiment")
    def sentiment_allowed(cls, value):
        allowed = {"POSITIVE", "NEGATIVE", "NEUTRAL", "IRRELEVANT", "HARMFUL", "SUGGESTIVE"}
        if value.upper() not in allowed:
            raise ValueError(f"Sentiment must be one of {allowed}")
        return value.upper()

parser = PydanticOutputParser(pydantic_object=SentimentAnalysis)

prompt = PromptTemplate(
    template="""
Analyze the user feedback below and respond with:
1. Sentiment (POSITIVE, NEGATIVE, NEUTRAL, IRRELEVANT, HARMFUL, or SUGGESTIVE),
2. Dominant emotion (e.g. Happy, Angry, Sad),
3. Rating from 1 to 5.
{format_instructions}
Feedback: {query}
""",
    input_variables=["query"],
    partial_variables={"format_instructions": parser.get_format_instructions()},
)

chain = prompt | model | parser

def analyze_feedbacks(text_input, file):
    feedbacks = []
    if file is not None:
        content = file.decode("utf-8")
        feedbacks = [line.strip() for line in content.strip().split("\n") if line.strip()]
    elif text_input.strip():
        feedbacks = [line.strip() for line in text_input.strip().split("\n") if line.strip()]

    if not feedbacks:
        return "Please provide some feedback.", None

    results = []
    for fb in feedbacks:
        try:
            result = chain.invoke({"query": fb})
            results.append({
                "Feedback": fb,
                "Sentiment": result.sentiment,
                "Emotion": result.emotion,
                "Rating": result.rating
            })
        except Exception as e:
            results.append({
                "Feedback": fb,
                "Sentiment": "Error",
                "Emotion": "Error",
                "Rating": "Error"
            })

    df = pd.DataFrame(results)

    fig, axs = plt.subplots(1, 2, figsize=(14, 5))

    sentiment_counts = df['Sentiment'].value_counts()
    axs[0].pie(sentiment_counts, labels=sentiment_counts.index, autopct='%1.1f%%', startangle=140)
    axs[0].set_title("Sentiment Distribution")

    try:
        rating_counts = df['Rating'].value_counts().sort_index()
        axs[1].bar(rating_counts.index.astype(str), rating_counts.values, color='skyblue')
        axs[1].set_title("Rating Frequency")
        axs[1].set_xlabel("Rating")
        axs[1].set_ylabel("Count")
    except:
        axs[1].text(0.5, 0.5, "Ratings not valid", ha='center', va='center')

    plt.tight_layout()
    return df, fig

with gr.Blocks(title="Feedback Sentiment Analyzer with Gemini") as demo:
    gr.Markdown("## Feedback Sentiment Analyzer using Gemini Flash")
    gr.Markdown("You can enter feedback manually (one per line) or upload a .txt file.")

    with gr.Row():
        text_box = gr.Textbox(label="Enter Feedback (One per Line)", lines=8, placeholder="The teacher was very helpful...")
        file_upload = gr.File(label="Upload a .txt File", file_types=[".txt"], type="binary")

    submit_btn = gr.Button("Analyze Feedback")

    output_table = gr.Dataframe(label="Results")
    output_plot = gr.Plot(label="Sentiment & Rating Charts")

    submit_btn.click(analyze_feedbacks, inputs=[text_box, file_upload], outputs=[output_table, output_plot])

demo.launch()


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.9/89.9 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.0/42.0 kB 303.8 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.6/8.6 MB 41.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.1/13.1 MB 34.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.5/46.5 MB 14.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 322.2/322.2 kB 10.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.3/11.3 MB 60.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.3/62.3 kB 2.4 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires pandas==

<ipython-input-1-072bdbd245df>:23: PydanticDeprecatedSince20: Pydantic V1 style `@validator` validators are deprecated. You should migrate to Pydantic V2 style `@field_validator` validators, see the migration guide for more details. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.11/migration/
  @validator("sentiment")


Running Gradio in a Colab notebook requires sharing enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://6dfaf4762aa1fbeb36.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
